# Predictive Maintenance
Data set: https://archive.ics.uci.edu/dataset/316/condition+based+maintenance+of+naval+propulsion+plants

In [1]:
import pandas as pd

In [2]:
columns = ['lp','v','GTT','GTn','GGn','Ts','Tp','T48','T1','T2','P48','P1','P2','Pexh','TIC','mf','kMc','kMt']
data = pd.read_csv('data/data.csv', names=columns, skiprows=1)

# kMc = GT Compressor decay state coefficient, kMt = GT Turbine decay state coefficient
data.head()

,lp,v,GTT,GTn,GGn,Ts,Tp,T48,T1,T2,P48,P1,P2,Pexh,TIC,mf,kMc,kMt
0,1.138,3,289.964,1349.489,6677.380,7.584,7.584,464.006,288,550.563,1.096,0.998,5.947,1.019,7.137,0.082,0.95,0.975
1,2.088,6,6960.180,1376.166,6828.469,28.204,28.204,635.401,288,581.658,1.331,0.998,7.282,1.019,10.655,0.287,0.95,0.975
2,3.144,9,8379.229,1386.757,7111.811,60.358,60.358,606.002,288,587.587,1.389,0.998,7.574,1.020,13.086,0.259,0.95,0.975
3,4.161,12,14724.395,1547.465,7792.630,113.774,113.774,661.471,288,613.851,1.658,0.998,9.007,1.022,18.109,0.358,0.95,0.975
4,5.140,15,21636.432,1924.313,8494.777,175.306,175.306,731.494,288,645.642,2.078,0.998,11.197,1.026,26.373,0.522,0.95,0.975


In [8]:
data.shape

(11934, 18)

In [3]:
data.describe().round(3)

,lp,v,GTT,GTn,GGn,Ts,Tp,T48,T1,T2,P48,P1,P2,Pexh,TIC,mf,kMc,kMt
count,11934.000,11934.000,11934.000,11934.000,11934.000,11934.000,11934.000,11934.000,11934.0,11934.000,11934.000,11934.000,11934.000,11934.000,11934.000,11934.000,11934.000,11934.000
mean,5.167,15.000,27247.499,2136.289,8200.947,227.336,227.336,735.495,288.0,646.215,2.353,0.998,12.297,1.029,33.641,0.662,0.975,0.988
std,2.626,7.746,22148.613,774.084,1091.316,200.496,200.496,173.681,0.0,72.676,1.085,0.000,5.337,0.010,25.841,0.507,0.015,0.008
min,1.138,3.000,253.547,1307.675,6589.002,5.304,5.304,442.364,288.0,540.442,1.093,0.998,5.828,1.019,0.000,0.068,0.950,0.975
25%,3.144,9.000,8375.884,1386.758,7058.324,60.317,60.317,589.873,288.0,578.092,1.389,0.998,7.447,1.020,13.678,0.246,0.962,0.981
50%,5.140,15.000,21630.659,1924.326,8482.082,175.268,175.268,706.038,288.0,637.142,2.083,0.998,11.092,1.026,25.276,0.496,0.975,0.988
75%,7.148,21.000,39001.427,2678.079,9132.606,332.365,332.365,834.066,288.0,693.924,2.981,0.998,15.658,1.036,44.552,0.882,0.988,0.994
max,9.300,27.000,72784.872,3560.741,9797.103,645.249,645.249,1115.797,288.0,789.094,4.560,0.998,23.140,1.052,92.556,1.832,1.000,1.000


### Training a simple MLP as a baseline to estimate compressor decay

In [20]:
import torch
import torch.nn as nn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split features and target
features = data.drop(columns=['kMc'])
target = data['kMc']

# Split data into train, validation, and test sets
train_size = 0.7
val_size = 0.15
test_size = 0.15

X_train, X_temp, y_train, y_temp = train_test_split(features, target, test_size=1 - train_size, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=test_size/(test_size + val_size), random_state=42)

In [21]:
# scale the input data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [22]:
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

input_dim = X_train_scaled.shape[1]
hidden_dim = 64  # Adjust this as needed
output_dim = 1
model = MLP(input_dim, hidden_dim, output_dim)

In [23]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [26]:
num_epochs = 1000  # Adjust the number of epochs as needed

for epoch in range(num_epochs):
    inputs = torch.tensor(X_train_scaled, dtype=torch.float32)
    targets = torch.tensor(y_train.values, dtype=torch.float32)

    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [100/1000], Loss: 0.0210
Epoch [200/1000], Loss: 0.0036
Epoch [300/1000], Loss: 0.0017
Epoch [400/1000], Loss: 0.0011
Epoch [500/1000], Loss: 0.0009
Epoch [600/1000], Loss: 0.0007
Epoch [700/1000], Loss: 0.0006
Epoch [800/1000], Loss: 0.0005
Epoch [900/1000], Loss: 0.0005
Epoch [1000/1000], Loss: 0.0004


In [27]:
model.eval()
with torch.no_grad():
    test_inputs = torch.tensor(X_test_scaled, dtype=torch.float32)
    test_targets = torch.tensor(y_test.values, dtype=torch.float32)
    test_outputs = model(test_inputs)

mse = nn.MSELoss()
rmse = torch.sqrt(mse(test_outputs, test_targets))
print(f'Root Mean Squared Error (RMSE): {rmse:.4f}')

Root Mean Squared Error (RMSE): 0.0204
